In [ ]:
from pandas import DataFrame, Series
import pandas as pd
pd.options.display.max_columns = 100

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['svg.fonttype'] = 'none'

In [ ]:
filename_2013 = 'data/Krankenhausverzeichnis_31_12_2013.xlsx'

xl_file_2013 = pd.ExcelFile(filename_2013)
df = xl_file_2013.parse('KHV_2013', skiprows=4)
df.head()

## Wieviele unterschiedliche psychiatrische Bettentypen gibt es?

In [ ]:
# Spalten an denen wir interessiert sind
columns = ['Kinderpsychiat', 'Psychiat', 'Psychiat_Sucht', 'Psychotherapeut']

# Konvertierung nach numerisch, fehlende Werte werden 0
df[columns] = df[columns].convert_objects(convert_numeric=True).fillna(0) 
df[columns].sum()

## Was sind die Top 10 Einrichtungen mit den meisten psychiatrischen Betten?

In [ ]:
# Summe über andere Achse
df['psych_summe'] = df[columns].sum(axis=1)
df['psych_summe']

In [ ]:
df.psych_summe.describe()

In [ ]:
df.sort_index(by='psych_summe', ascending=False).head(10)

## Welcher Kreis hat die meisten psychiatrischen Betten pro Einwohner?

Erst einmal Daten besorgen: https://www.destatis.de/DE/ZahlenFakten/GesellschaftStaat/Bevoelkerung/Bevoelkerungsstand/Bevoelkerungsstand.html#Tabellen

In [ ]:
population = pd.read_csv('data/12411-0014.csv', skiprows=5, skipfooter=5, delimiter=';', encoding='latin-1')
population.head()

In [ ]:
# Umbenennen von Spalten
population.rename(columns={'Unnamed: 0': 'AGS', 'Unnamed: 1': 'Kreisname'}, inplace=True)
years = list(range(1996, 2015))
year_renames = {'31.12.%d' % (year - 1): year for year in years}
population.rename(columns=year_renames, inplace=True)

# Konvertieren nach numerisch
population[years] = population[years].convert_objects(convert_numeric=True)
population.head()

Wie führt man die psychiatrischen Betten und die Bevölkerungszahlen zusammen?
Mit dem Amtlischen Gemeindeschlüssel: https://de.wikipedia.org/wiki/Amtlicher_Gemeindeschl%C3%BCssel

In [ ]:
population['AGS'] = population['AGS'].apply(lambda x: '{:0>5d}'.format(x))
population = population.set_index('AGS')

pop2013 = population[[2013]]
pop2013.head()

Wo ist der AGS bei unseren psychiatrischen Betten?

In [ ]:
df.head(1)

In [ ]:
def make_ags(x):
    return '{:0>2d}{:0>3d}'.format(x.Land, 0 if x.Land == 11 else x.Kreis)

df['AGS'] = df.apply(make_ags, axis=1)
df.head(1)

In [ ]:
df = df.merge(pop2013.reset_index(), on='AGS')
df.head()

In [ ]:
PER_POPULATION = 100000
df['psych_pro_einwohner'] = df['psych_summe'] / df[2013] * PER_POPULATION
df['psych_pro_einwohner']

In [ ]:
df.psych_pro_einwohner.describe()

In [ ]:
df.sort_index(by='psych_pro_einwohner', ascending=False).head(10)

Bei Kreisen sollte man den geographischen Kontext kennen.

In [ ]:
import folium


geo_path = 'data/kreise.topo.json'
key = 'psych_pro_einwohner'

geo_data = df.copy()
geo_data = geo_data[geo_data[key] != 0]

min_val = geo_data[key].min()
q0 = geo_data[key].quantile( .05)
q1 = geo_data[key].quantile( .25)
q2 = geo_data[key].quantile( .5)
q3 = geo_data[key].quantile( .75)
max_val = geo_data[key].max()

SCALE_COUNT = 5
scale = list(geo_data[key].quantile([x / SCALE_COUNT for x in range(SCALE_COUNT)]).values)
scale = [min_val, q1, q2, q3, max_val]

m = folium.Map(location=[52, 10], zoom_start=6, height=800)
m.geo_json(geo_path=geo_path, data=geo_data, data_out='%s_map.json' % key,
             columns=['AGS', key],
             threshold_scale=scale,
             key_on='feature.properties.AGS',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Betten per 100 000',
             topojson='objects.kreise'
)
m.create_map(path='map.html')
m

## Wie ist das Verhältnis zwischen der Anzahl der psychiatrischen Betten und Gesamtbetten?

In [ ]:
df.Betten_Ins = df.Betten_Ins.convert_objects(convert_numeric=True)
plt.scatter(df.Betten_Ins, df.psych_summe)

## Wo ist der Anteil an psychiatrischen an allen Betten am höchsten?